<a href="https://colab.research.google.com/github/ngobaongoc61/DN-59/blob/master/Process_and_Thread.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **0. Write and Combine C program**

In [ ]:
%%writefile sample.cpp

#include <stdio.h>
#define NAME "LUCAS"

int main()
{
    printf("Hello world, %s!", NAME); // In dong chu "Hello world"

    return 0;
}

Overwriting sample.cpp


Compile file sample.cpp and then run it using shell command.

In [ ]:
!g++ sample.cpp -o sample
!./sample

Hello world, LUCAS!

**Example about command line arguments in C**

In [ ]:
%%writefile test_arg.cpp

#include <stdio.h>
#include <stdlib.h>

int main(int argc, char* argv[])
{
    if (argc < 2)
    {
        printf("We cannot find the name from command line!");
    }
    else
        printf("Hello world, %s %s\n", argv[1], argv[2]);
    return 0;
}

Overwriting test_arg.cpp


In [ ]:
!g++ test_arg.cpp -o test_arg
!./test_arg

We cannot find the name from command line!

In [ ]:
!./test_arg "Lucas Tran"

Hello world, Lucas Tran (null)


# Làm sao để chạy code C?

## Chạy chương trình C cơ bản

Để chạy code C, chúng ta cần thực hiện các bước như sau:


1.   Tạo ra file source code thông qua magic cell `%%writefile`
2.   Biên dịch file source code, để chuyển từ source code thành file khả thực thi, sử dụng câu lệnh `gcc` đối với file C và `g++` đối với file C++
3.   Thực thi file khả thực thi thông qua địa chỉ của file



In [ ]:
%%writefile sample.cpp

#include<stdio.h>

int main()
{
    printf("Hello world, I am Tran Hoang Loc");

    return 0;
}

Overwriting sample.cpp


In [ ]:
!g++ sample.cpp -o sample

In [ ]:
!./sample

Hello world, I am Tran Hoang Loc

## Truyền tham số dòng lệnh cho chương trình C

Tham số của chương trình C được nhận thông qua 2 thành phần:


*   **argc**: viết tắt của argument count, tức là số tham số dòng lệnh được truyền vào chương trình
*   **argv[]**: mảng lưu các tham số dòng lệnh
*   Tham số dòng lệnh được nhận từ hàm main theo cấu trúc khai báo sau `int main(int argc, char* argv[])`



In [ ]:
%%writefile test_arg.cpp

#include<stdio.h>

int main(int argc, char* argv[])
{
    printf("So luong tham so tren dong lenh: %d\n", argc);

    for (int i = 0; i < argc; i++)
        printf("argv[%d]: %s\n", i, argv[i]);

    return 0;
}

Overwriting test_arg.cpp


In [ ]:
!g++ test_arg.cpp -o test_arg
!./test_arg

So luong tham so tren dong lenh: 1
argv[0]: ./test_arg


`argv[0]` (phần tử đầu tiên trong mảng `argv[]`) chính là tên chương trình

In [ ]:
!./test_arg "Tran Hoang Loc"

So luong tham so tren dong lenh: 2
argv[0]: ./test_arg
argv[1]: Tran Hoang Loc


Lưu ý: tham số dòng lệnh được truyền theo dạng chuỗi (string), do đó, nếu cần xử lý dạng số thì cần phải chuyển từ chuỗi về số trước.

# **1. Process Creation**


## **1.1. Create process with `fork()`**

In [ ]:
%%writefile test_fork.cpp

#include <stdio.h>
#include <stdlib.h>
#include <unistd.h>
#include <sys/wait.h>
#include <sys/types.h>

int main(int argc, char* argv[])
{
    __pid_t pid;
    pid = fork();

    if (pid > 0)
    {
        printf("PARENTS | PID = %ld | PPID = %ld\n",
        (long)getpid(), (long)getppid());

        if (argc > 2)
        {
            printf("PARENTS | There are %d arguments\n", argc - 1);
            wait(NULL);
        }
    }
    if (pid == 0)
    {
        printf("CHILDREN | PID = %ld | PPID = %ld\n",
        (long)getpid(), (long)getppid());

        printf("CHILDREN | List of arguments: \n");

        for (int i = 1; i < argc; i++)
        {
            printf("%s\n", argv[i]);
        }
    }

    return 0;
}

Overwriting test_fork.cpp


In [ ]:
!g++ test_fork.cpp -o test_fork
!./test_fork ThamSo1 ThamSo2 ThamSo3

PARENTS | PID = 87192 | PPID = 26787
PARENTS | There are 3 arguments
CHILDREN | PID = 87193 | PPID = 87192
CHILDREN | List of arguments: 
ThamSo1
ThamSo2
ThamSo3


## **1.2. Create process with `exec()`**



**Firstly, create `count.sh` which do following tasks:**

*   Print process name
*   Print PPID (parent's process ID)
*   Count from 1 to $1 while printing numbers to `count.txt`

*Notes: the command `ps -ef | grep count.sh` lists all processes and then find which line (process' info) contains string "count.sh". This command helps us find information (PID and PPID) of process `./count.sh`*



In [ ]:
%%writefile count.sh

#!/bin/bash

echo "Implementing: $0"
echo "PPID of count.sh: "
ps -ef | grep count.sh

i=1

while [ $i -le $1 ]
do
    echo $i >> count.txt
    i=$((i + 1))
    sleep 1
done

exit 0

Writing count.sh


We have to grant execution permission to count.sh in order to execute it. We only do this once.

In [ ]:
!chmod 755 count.sh

Run the script (with $1 = 20) and check result if count.txt is created/updated.

In [ ]:
!./count.sh 20

Implementing: ./count.sh
PPID of count.sh: 
root       89839   26787  0 08:48 ?        00:00:00 /bin/bash -c ./count.sh 20
root       89841   89839  0 08:48 ?        00:00:00 grep count.sh


**Create `test_execl.cpp`, compile and run it**

In [ ]:
%%writefile test_execl.cpp

#include <stdio.h>
#include <stdlib.h>
#include <unistd.h>
#include <sys/wait.h>
#include <sys/types.h>

int main(int argc, char* argv[])
{
    __pid_t pid;
    pid = fork();
    if (pid > 0)
    {
        printf("PARENTS | PID = %ld | PPID = %ld\n",
        (long)getpid(), (long)getppid());

        if (argc > 2)
            printf("PARENTS | There are %d arguments\n",
            argc - 1);
        wait(NULL);
    }
    if (pid == 0)
    {
        execlp("./count.sh", "./count.sh", "10", NULL);

        printf("CHILDREN | PID = %ld | PPID = %ld\n",
        (long)getpid(), (long)getppid());

        printf("CHILDREN | List of arguments: \n");

        for (int i = 1; i < argc; i++)
        {
            printf("%s\n", argv[i]);
        }
    }

    return 0;
}


Overwriting test_execl.cpp


In [ ]:
!g++ test_execl.cpp -o test_execl
!./test_execl ThamSo1 ThamSo2

PARENTS | PID = 90859 | PPID = 26787
PARENTS | There are 2 arguments
Implementing: ./count.sh
PPID of count.sh: 
root       90860   90859  0 08:53 ?        00:00:00 /bin/sh ./count.sh 10
root       90862   90860  0 08:53 ?        00:00:00 grep count.sh


## **1.3. Create process with `system()`**

**Create `test_system.cpp`, compile and run it**

In [ ]:
%%writefile test_system.cpp

#include <stdio.h>
#include <stdlib.h>
#include <unistd.h>
#include <sys/wait.h>
#include <sys/types.h>

int main(int argc, char* argv[])
{
    printf("PARENTS | PID = %ld | PPID = %ld\n",
    (long)getpid(), (long)getppid());

    if (argc > 2)
        printf("PARENTS | There are %d arguments\n", argc
        - 1);

    system("./count.sh 10");

    printf("PARENTS | List of arguments: \n");

    for (int i = 1; i < argc; i++)
    {
        printf("%s\n", argv[i]);
    }

    return 0;
}

Writing test_system.cpp


In [ ]:
!g++ test_system.cpp -o test_system
!./test_system ThamSo1 ThamSo2 ThamSo3

PARENTS | PID = 575 | PPID = 225
PARENTS | There are 3 arguments
Implementing: ./count.sh
PPID of count.sh: 
root         576     575  0 14:00 ?        00:00:00 sh -c ./count.sh 10
root         577     576  0 14:00 ?        00:00:00 /bin/sh ./count.sh 10
root         579     577  0 14:00 ?        00:00:00 grep count.sh
PARENTS | List of arguments: 
ThamSo1
ThamSo2
ThamSo3


## **BÀI TẬP**

### **Bài tập 1.** Hãy giải thích cơ chế tạo tiến trình của hàm `fork()`, `exec()`, và `system()`


### **Bài tập 2.** So sánh sự giống và khác nhau của 3 cách tạo tiến trình trên

# **2. Working with Threads**

## **2.1. Thread creation**

In [ ]:
%%writefile thread_creation.cpp

#include <pthread.h>
#include <stdio.h>

void *thread_print(void *message)
{
    while(1)
        printf("Hello, how are you?\n");
}

int main()
{
    pthread_t idthread;

    pthread_create(
        &idthread,
        NULL,
        &thread_print,
        NULL
    );

    while(1)
        printf("I'm fine, and you?\n");

    return 0;
}

Writing thread_creation.cpp


In [ ]:
!gcc thread_creation.cpp -o thread_creation
!./thread_creation

Streaming output truncated to the last 5000 lines.
Hello, how are you?
Hello, how are you?
Hello, how are you?
Hello, how are you?
Hello, how are you?
Hello, how are you?
Hello, how are you?
Hello, how are you?
Hello, how are you?
Hello, how are you?
Hello, how are you?
Hello, how are you?
Hello, how are you?
Hello, how are you?
Hello, how are you?
Hello, how are you?
Hello, how are you?
Hello, how are you?
Hello, how are you?
Hello, how are you?
Hello, how are you?
Hello, how are you?
Hello, how are you?
Hello, how are you?
Hello, how are you?
Hello, how are you?
Hello, how are you?
Hello, how are you?
Hello, how are you?
Hello, how are you?
Hello, how are you?
Hello, how are you?
Hello, how are you?
Hello, how are you?
Hello, how are you?
Hello, how are you?
Hello, how are you?
Hello, how are you?
Hello, how are you?
Hello, how are you?
Hello, how are you?
Hello, how are you?
Hello, how are you?
Hello, how are you?
Hello, how are you?
Hello, how are you?
Hello, how are you?
Hello, ho

## **2.2. Stop threads**

In [ ]:
%%writefile thread_selfexit.c

#include <pthread.h>
#include <stdio.h>
#include <stdlib.h>
#include <unistd.h>

#define NUM_THREADS 2

void *thread_print(void *threadid)
{
    long tid;
    tid = (long)threadid;
    printf("Hello IT007! I'm Thread #%ld!\n", tid);
    pthread_exit(NULL);
}

int main()
{
    pthread_t threads[NUM_THREADS];
    int check;
    long tID;

    for(tID = 0; tID < NUM_THREADS; tID++)
    {
        check = pthread_create(
                    &threads[tID],
                    NULL,
                    &thread_print,
                    (void *)tID
                );

        if (check != 0)
        {
            printf("[MAIN_THREAD] ERROR! Can't create Thread #%ld\n", tID);
            exit(-1);
        }
    }


    sleep(10);

    printf("[MAIN THREAD] Bye bye!\n");
    pthread_exit(NULL);
    return 0;
}

Writing thread_selfexit.c


In [ ]:
!gcc thread_selfexit.c -o thread_selfexit
!./thread_selfexit

Hello IT007! I'm Thread #1!
Hello IT007! I'm Thread #0!
[MAIN THREAD] Bye bye!


## **2.3. Merge threads**

In [ ]:
%%writefile thread_join.c

#include <pthread.h>
#include <stdio.h>
#include <stdlib.h>
#include <unistd.h>

#define NUM_THREADS 2

void *thread_print(void *threadid)
{
    long tid;
    tid = (long)threadid;
    printf("Hello IT007! I'm Thread #%ld!\n", tid);
    sleep(10);
}

int main()
{
    pthread_t threads[NUM_THREADS];
    int check;
    long tID;

    for(tID = 0; tID < NUM_THREADS; tID++)
    {
        printf("[MAIN THREAD] Create Thread #%ld\n", tID);
        check = pthread_create(
                    &threads[tID],
                    NULL,
                    &thread_print,
                    (void *)tID
                );

        if (check != 0)
        {
            printf("[MAIN_THREAD] ERROR! Can't create Thread #%ld\n", tID);
            exit(-1);
        }

        pthread_join(threads[tID], NULL);
    }

    printf("[MAIN THREAD] Bye bye!\n");
    pthread_exit(NULL);
    return 0;
}

Overwriting thread_join.c


In [ ]:
!gcc thread_join.c -o thread_join
!./thread_join

[MAIN THREAD] Create Thread #0
Hello IT007! I'm Thread #0!
[MAIN THREAD] Create Thread #1
Hello IT007! I'm Thread #1!
[MAIN THREAD] Bye bye!


## **2.4. Send arguments to thread**

In [ ]:
%%writefile thread_args.c

#include <pthread.h>
#include <stdio.h>
#include <stdlib.h>
#include <unistd.h>

struct print_params
{
    char character;
    int count;
};

void *char_print(void* args)
{
    struct print_params *p = (struct print_params *)args;
    int i;

    for (i = 0; i < p->count; i++)
        printf("%c\n", p->character);

    return NULL;
}

int main()
{
    pthread_t tid;
    struct print_params th_args;

    th_args.character = 'X';
    th_args.count = 5;

    pthread_create(
        &tid,
        NULL,
        &char_print,
        &th_args
    );

    pthread_join(tid, NULL);

    pthread_exit(NULL);
    return 0;
}

Overwriting thread_args.c


In [ ]:
!gcc thread_args.c -o thread_args
!./thread_args

X
X
X
X
X


# **3. Đồng bộ tiểu trình**

## **3.1. Mutex Locks**

Cho 2 tiến trình (hoặc tiểu trình) cùng thực hiện một công việc như bên dưới:

```
x = x + 1;
if (x == 20)
    x = 0;
```

In [ ]:
%%writefile mutex_example.cpp

#include <pthread.h>
#include <stdio.h>

int x = 0; // Bien toan cuc, duoc chia se giua A va B
pthread_mutex_t mut; // Khai bao mutex toan cuc

void *ProcessA(void *message)
{
    while(1)
    {
        pthread_mutex_lock(&mut); // Lock mutex
        x = x + 1;
        if (x == 20)
            x = 0;
        printf("Process A | x = %d\n", x);
        pthread_mutex_unlock(&mut); // Unlock mutex
    }
}

void *ProcessB(void *message)
{
    while(1)
    {
        pthread_mutex_lock(&mut); // Lock mutex
        x = x + 1;
        if (x == 20)
            x = 0;
        printf("Process B | x = %d\n", x);
        pthread_mutex_unlock(&mut); // Unlock mutex
    }
}

int main()
{
    pthread_t idthreadA, idthreadB;
    pthread_mutex_init(&mut, NULL); // Khoi tao mutex

    pthread_create(
        &idthreadA,
        NULL,
        &ProcessA,
        NULL
    );

    pthread_create(
        &idthreadB,
        NULL,
        &ProcessB,
        NULL
    );

    while(1){}

    return 0;
}

Overwriting mutex_example.cpp


In [ ]:
!g++ mutex_example.cpp -o mutex_example

In [ ]:
!./mutex_example

Streaming output truncated to the last 5000 lines.
Process B | x = 6
Process B | x = 7
Process B | x = 8
Process B | x = 9
Process B | x = 10
Process B | x = 11
Process B | x = 12
Process B | x = 13
Process B | x = 14
Process B | x = 15
Process B | x = 16
Process B | x = 17
Process B | x = 18
Process B | x = 19
Process B | x = 0
Process B | x = 1
Process B | x = 2
Process B | x = 3
Process B | x = 4
Process B | x = 5
Process B | x = 6
Process B | x = 7
Process B | x = 8
Process B | x = 9
Process B | x = 10
Process B | x = 11
Process A | x = 12
Process A | x = 13
Process A | x = 14
Process A | x = 15
Process A | x = 16
Process A | x = 17
Process A | x = 18
Process A | x = 19
Process A | x = 0
Process A | x = 1
Process A | x = 2
Process A | x = 3
Process A | x = 4
Process A | x = 5
Process A | x = 6
Process A | x = 7
Process A | x = 8
Process A | x = 9
Process A | x = 10
Process A | x = 11
Process A | x = 12
Process A | x = 13
Process A | x = 14
Process A | x = 15
Process A | x = 16
Proc

## **3.2. Semaphores**

Cho 02 tiến trình Produce và Consume như sau:
```
Produce()
{
    products++;
}
```
```
Consume()
{
    sells++;
}
```
Ta có điều kiện là tại mọi thời điểm `products >= sells`
Do sells chỉ được phép tăng khi `sells < products`, và được tăng tối đa một lượng bằng `products - sells`. Gọi semaphore `stock` được khởi tạo bằng `products - sells`.

In [ ]:
%%writefile semaphore_example.cpp

#include <pthread.h>
#include <stdio.h>
#include <semaphore.h>

int products = 0, sells = 0; // Bien toan cuc, duoc chia se giua A va B
sem_t stock; // Khai bao semaphore la toan cuc

void *Produce(void *message)
{
    while(1)
    {
        products++;
        printf("Produce | products = %d\n", products);
        sem_post(&stock); // Thao tac signal semaphore
    }
}

void *Consume(void *message)
{
    while(1)
    {
        sem_wait(&stock); // Thao tac wait semaphore
        sells++;
        printf("Consume | sells = %d\n", sells);
    }
}

int main()
{
    pthread_t idthreadA, idthreadB;
    sem_init(&stock, 0, products - sells); // Gia tri khoi tao cua stock la products - sells

    pthread_create(
        &idthreadA,
        NULL,
        &Produce,
        NULL
    );

    pthread_create(
        &idthreadB,
        NULL,
        &Consume,
        NULL
    );

    while(1){}

    return 0;
}

Overwriting semaphore_example.cpp


In [ ]:
!g++ semaphore_example.cpp -o semaphore_example

In [ ]:
!./semaphore_example

Streaming output truncated to the last 5000 lines.
Consume | sells = 67536
Consume | sells = 67537
Consume | sells = 67538
Consume | sells = 67539
Consume | sells = 67540
Consume | sells = 67541
Consume | sells = 67542
Consume | sells = 67543
Consume | sells = 67544
Consume | sells = 67545
Consume | sells = 67546
Consume | sells = 67547
Consume | sells = 67548
Consume | sells = 67549
Consume | sells = 67550
Consume | sells = 67551
Consume | sells = 67552
Consume | sells = 67553
Consume | sells = 67554
Consume | sells = 67555
Consume | sells = 67556
Consume | sells = 67557
Consume | sells = 67558
Consume | sells = 67559
Consume | sells = 67560
Consume | sells = 67561
Consume | sells = 67562
Consume | sells = 67563
Consume | sells = 67564
Consume | sells = 67565
Consume | sells = 67566
Consume | sells = 67567
Consume | sells = 67568
Consume | sells = 67569
Consume | sells = 67570
Consume | sells = 67571
Consume | sells = 67572
Consume | sells = 67573
Consume | sells = 67574
Consume | sel